### Create model 1 & 2 dataset

In [2]:
import pandas as pd
import os
import shutil

# Load the metadata
metadata = pd.read_csv('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/metadata/UrbanSound8K.csv')

# Make the new directory if it doesn't exist
os.makedirs('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2', exist_ok=True)

# Create directories for each class in model2
for class_name in metadata['class'].unique():
    os.makedirs(f'C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2/{class_name}', exist_ok=True)

# The new CSV for the 'model2' subset
model2_data = pd.DataFrame()

# Select the files for each class
for class_id in metadata['classID'].unique():
    class_data = metadata[metadata['classID'] == class_id]
    selected_files = class_data.sample(frac=0.1, random_state=42)
    model2_data = model2_data.append(selected_files)

    # Move the selected files into the 'model2' directory
    for index, row in selected_files.iterrows():
        original_path = f"C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/fold{row['fold']}/{row['slice_file_name']}"
        new_path = f"C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2/{row['class']}/{row['slice_file_name']}"
        shutil.move(original_path, new_path)

# Save the model2_data into a new CSV file
model2_data.to_csv('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/metadata/model2.csv', index=False)

# Removing model2 data from the original dataframe to create the model1 data
model1_data = metadata.drop(model2_data.index)

# Save the model1_data into a new CSV file
model1_data.to_csv('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/metadata/model1.csv', index=False)


### Rearrange folders with class

In [ ]:
import os
import shutil
import pandas as pd

audio_dir = 'C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/'
csv_file = 'C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/metadata/UrbanSound8K.csv'

data = pd.read_csv(csv_file)

for index, row in data.iterrows():
    class_name = row['class']
    file_name = row['slice_file_name']
    
    target_dir = os.path.join(audio_dir, class_name)
    os.makedirs(target_dir, exist_ok=True)
    
    source_path = os.path.join(audio_dir, f"fold{row['fold']}", file_name)
    destination_path = os.path.join(target_dir, file_name)
    
    shutil.copy(source_path, destination_path)


### Create model2 train files

In [3]:
import random
import pandas as pd
from pydub import AudioSegment

audio_dir = 'C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2'
csv_path = 'C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/metadata/model2.csv'
output_dir = 'C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/output/model2_audio_clips/'
df = pd.read_csv(csv_path)
os.makedirs(output_dir, exist_ok=True)


class_mapping = {
    0: 'air_conditioner',
    1: 'car_horn',
    2: 'children_playing',
    3: 'dog_bark',
    4: 'drilling',
    5: 'engine_idling',
    6: 'gun_shot',
    7: 'jackhammer',
    8: 'siren',
    9: 'street_music'
}

behavior_patterns = {
    'Dog1': {'sound': [1], 'bark_prob': [1.0]},
    'Dog2': {'sound': [2], 'bark_prob': [1.0]},
    'Dog3': {'sound': [5], 'bark_prob': [1.0]},
    'Dog4': {'sound': [7, 6], 'bark_prob': [0.1, 0.9]},
    'Dog5': {'sound': [9, 1], 'bark_prob': [0.9, 0.1]},
    'Dog6': {'sound': [2, 8], 'bark_prob': [0.2, 0.8]},
    'Dog7': {'sound': [4, 0, 5], 'bark_prob': [0.4, 0.3, 0.3]},
    'Dog8': {'sound': [7, 9, 0], 'bark_prob': [0.3, 0.2, 0.5]},
    'Dog9': {'sound': [5, 1, 8], 'bark_prob': [0.8, 0.1, 0.1]},
    'Dog10': {'sound': [0, 5, 2], 'bark_prob': [0.4, 0.2, 0.4]}
}

sound_duration = 1.0
gap_duration = 1.0 
summary_data = []
file_names = []

#  for each dog
for dog, behavior in behavior_patterns.items():
    dog_sounds = behavior['sound']
    bark_prob = behavior['bark_prob']

    random_df = df[~df['classID'].isin([3] + dog_sounds)].reset_index(drop=True)


    # find dog bark
    bark_df = df[df['classID'] == 3].reset_index(drop=True)
    valid_file_found = False
    while not valid_file_found:
        try:
            bark_sound_file = bark_df['slice_file_name'].sample(1).values[0]
            bark_sound_path = os.path.join(audio_dir, class_mapping[3], bark_sound_file)
            bark_sound = AudioSegment.from_wav(bark_sound_path)
            valid_file_found = True
        except Exception as e:
            print(f"Error loading sound file: {bark_sound_path}")
            print(f"Exception: {e}")
            bark_df = bark_df[bark_df['slice_file_name'] != bark_sound_file]  
            if bark_df.empty:
                break 

    if valid_file_found:
        for clip_index in range(10):
            audio_clip = AudioSegment.silent(duration=0)
            clip_summary = []

            # random sounds
            for _ in range(8):  
                    available_sounds = random_df['slice_file_name'].tolist()
                    if len(available_sounds) > 0:
                        sound_clip = random.choice(available_sounds)
                        available_sounds.remove(sound_clip)

                        sound_path = os.path.join(audio_dir, class_mapping[random_df.loc[random_df['slice_file_name'] == sound_clip, 'classID'].values[0]], sound_clip)

                        try:
                            # Load the sound clip
                            sound = AudioSegment.from_wav(sound_path)
                            audio_clip += sound
                            audio_clip += AudioSegment.silent(duration=int(gap_duration * 1000))
                            clip_summary.append(random_df.loc[random_df['slice_file_name'] == sound_clip, 'classID'].values[0])
                        except Exception as e:
                            print(f"Error loading sound file: {sound_path}")
                            print(f"Exception: {e}")
                            continue

            # behavior pattern
            sound_prob = random.choices(range(len(dog_sounds)), weights=bark_prob, k=1)[0]
            behavior_sound = dog_sounds[sound_prob]
            behavior_class_label = class_mapping[behavior_sound]
            behavior_df = df[df['classID'] == behavior_sound].reset_index(drop=True)

            if len(behavior_df) > 0:
                sound_file = behavior_df['slice_file_name'].sample(1).values[0]
                sound_path = os.path.join(audio_dir, behavior_class_label, sound_file)
                try:
                    sound = AudioSegment.from_wav(sound_path)

                    audio_clip += sound
                    clip_summary.append(behavior_sound)

                    # add gap
                    gap = AudioSegment.silent(duration=int(gap_duration * 1000))
                    audio_clip += gap
                except Exception as e:
                        print(f"Error loading sound file: {sound_path}")
                        print(f"Exception: {e}")
                        continue


            # dog bark 
            audio_clip += bark_sound
            clip_summary.append(3)

            # export 
            file_name = f'{dog}_{clip_index + 1}.wav'
            output_path = os.path.join(output_dir, file_name)
            audio_clip.export(output_path, format='wav')

            summary_data.append(clip_summary)
            file_names.append(file_name)

summary_df = pd.DataFrame(summary_data, columns=[f'Sound_{i+1}' for i in range(10)])
summary_df['File_Name'] = file_names

# save csv
summary_path = os.path.join(output_dir, 'summary.csv')
summary_df.to_csv(summary_path, index=False)

c:\Users\zzzl0\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\zzzl0\anaconda3\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


Error loading sound file: C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2\jackhammer\203929-7-3-5.wav
Exception: [WinError 2] 系统找不到指定的文件。
Error loading sound file: C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2\jackhammer\203929-7-2-2.wav
Exception: [WinError 2] 系统找不到指定的文件。
Error loading sound file: C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2\jackhammer\203929-7-7-15.wav
Exception: [WinError 2] 系统找不到指定的文件。
Error loading sound file: C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio/model2\engine_idling\176787-5-0-21.wav
Exception: [WinError 2] 系统找不到指定的文件。
Error loading sound file: C:/Users/zzzl0/Desktop/predicting-and-avoidin